In [4]:
# #preprocessing
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('words')
# nltk.download('omw-1.4')
# #detecting non-english
# !pip install langdetect
# !pip install langid
# #correcting spelling
# !pip install jedi
# !pip install pyspellchecker
# !pip install autocorrect
# !pip install pyenchant
# !pip install transformers

In [5]:
import pandas as pd

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words as nltk_words
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

from langdetect import detect
import langid

import autocorrect
from spellchecker import SpellChecker


import plotly.graph_objs as go

import torch
from transformers import BertTokenizer, BertModel

from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from wordcloud import WordCloud

import matplotlib.pyplot as plt
import seaborn as sns
import pickle

%matplotlib inline

In [6]:
df = pd.read_csv("test_CCS.csv")

In [7]:
df.head()

,job title,office location,department,salary offered for the job,company information,job description,job requirements,benefits,telecommuting,company logo exist?,has_questions,employment_type,experience required,education required,industry,function,fake?
0,Commercial/Industrial Construction Superintendent,"US, WA, Skagit County",NaN,NaN,NaN,Seeking qualified candidates for a full-time s...,Minimum five years of construction management ...,"401K, company vehicle, health, vision and dent...",0,0,1,Full-time,NaN,Unspecified,Construction,Management,no
1,Title Closer / Settlement Agent,"US, VA, Richmond",NaN,NaN,Network Closing Services has been serving Lend...,Looking for a change? Not happy where you are...,Are you a top Closer in your market? Are you a...,"We provide a fun and rewarding atmosphere, our...",0,1,1,Full-time,Mid-Senior level,NaN,Real Estate,NaN,no
2,Back-end Developer,"NL, NH, Amsterdam",IT,NaN,About the CompanyWe are ticketscript - the Eur...,About the CompanyWe are ticketscript - the Eur...,Your profileThe basics- 5+ years of experience...,Our offerA competitive salary within an intern...,0,1,0,Full-time,Associate,NaN,Computer Software,Information Technology,no
3,Full-time Web Developer,"PH, ,",NaN,528000-636000,NaN,#NAME?,- Must have extensive knowledge of CMS framewo...,NaN,0,1,0,Full-time,Not Applicable,Unspecified,Information Technology and Services,Information Technology,no
4,Program Manager,"US, CA, Mountain View",NaN,NaN,"Come be a part of one of the fastest growing, ...",We are looking for a Program Manager who can h...,Desired skills…10+ years of industry experienc...,Competitive base salaryStock OptionsFull benef...,0,1,1,Full-time,NaN,NaN,NaN,NaN,no


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3198 entries, 0 to 3197
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   job title                   3198 non-null   object
 1   office location             3153 non-null   object
 2   department                  1115 non-null   object
 3   salary offered for the job  505 non-null    object
 4   company information         2621 non-null   object
 5   job description             3197 non-null   object
 6   job requirements            2729 non-null   object
 7   benefits                    1976 non-null   object
 8   telecommuting               3198 non-null   int64 
 9   company logo exist?         3198 non-null   int64 
 10  has_questions               3198 non-null   int64 
 11  employment_type             2627 non-null   object
 12  experience required         1973 non-null   object
 13  education required          1792 non-null   obje

In [9]:
#CHECKING the precentage of NULL VALUES
nulls=df.isnull().sum()
nulls

job title                        0
office location                 45
department                    2083
salary offered for the job    2693
company information            577
job description                  1
job requirements               469
benefits                      1222
telecommuting                    0
company logo exist?              0
has_questions                    0
employment_type                571
experience required           1225
education required            1406
industry                       837
function                      1122
fake?                            0
dtype: int64

In [10]:
df["employment_type"].fillna(value="Other", inplace=True)
df["experience required"].fillna(value="Not Applicable", inplace=True)
df["education required"].fillna(value="Unspecified", inplace=True)

In [11]:
df.fillna(value="not mentioned", inplace=True)

In [12]:
#CHECKING the precentage of NULL VALUES
nulls=df.isnull().sum()
nulls

job title                     0
office location               0
department                    0
salary offered for the job    0
company information           0
job description               0
job requirements              0
benefits                      0
telecommuting                 0
company logo exist?           0
has_questions                 0
employment_type               0
experience required           0
education required            0
industry                      0
function                      0
fake?                         0
dtype: int64

In [13]:
df["Full job"] = "job title: " + df["job title"] + " - the location of the company is " + df["office location"] + " - the depatment you will work at is " + df["department"] + " - the salary offered is " + df["salary offered for the job"] + " - and here is the company information: " + df["company information"] + " - the job description: " +df["job description"] + " - and here are the requirments of the job: " + df["job requirements"] + " - the benfits you will gain: " + df["benefits"] + " - industry: " + df["industry"] + " - you funtion: " + df["function"]

In [14]:
#incoding fake? column
df["fake?"].replace(to_replace="no",value=0,inplace=True)
df["fake?"].replace(to_replace="yes",value=1,inplace=True)

In [15]:
df.drop("job title" ,axis = 1 , inplace = True)
df.drop("office location" ,axis = 1 , inplace = True)
df.drop("department" ,axis = 1 , inplace = True)
df.drop("salary offered for the job" ,axis = 1 , inplace = True)
df.drop("company information" ,axis = 1 , inplace = True)
df.drop("job description" ,axis = 1 , inplace = True)
df.drop("job requirements" ,axis = 1 , inplace = True)
df.drop("benefits" ,axis = 1 , inplace = True)
df.drop("industry" ,axis = 1 , inplace = True)
df.drop("function" ,axis = 1 , inplace = True)

In [16]:
df["employment_type"].replace(to_replace="Other",value=0,inplace=True)
df["employment_type"].replace(to_replace="Temporary",value=1,inplace=True)
df["employment_type"].replace(to_replace="Part-time",value=2,inplace=True)
df["employment_type"].replace(to_replace="Full-time",value=3,inplace=True)
df["employment_type"].replace(to_replace="Contract",value=4,inplace=True)

In [17]:
df["experience required"].replace(to_replace="Not Applicable",value=0,inplace=True)
df["experience required"].replace(to_replace="Internship",value=1,inplace=True)
df["experience required"].replace(to_replace="Entry level",value=2,inplace=True)
df["experience required"].replace(to_replace="Mid-Senior level",value=3,inplace=True)
df["experience required"].replace(to_replace="Associate",value=4,inplace=True)
df["experience required"].replace(to_replace="Executive",value=5,inplace=True)
df["experience required"].replace(to_replace="Director",value=6,inplace=True)

In [18]:
label_encoder = pickle.load(open('encoder', 'rb'))
df['education required']= label_encoder.transform(df['education required'])

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3198 entries, 0 to 3197
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   telecommuting        3198 non-null   int64 
 1   company logo exist?  3198 non-null   int64 
 2   has_questions        3198 non-null   int64 
 3   employment_type      3198 non-null   int64 
 4   experience required  3198 non-null   int64 
 5   education required   3198 non-null   int64 
 6   fake?                3198 non-null   int64 
 7   Full job             3198 non-null   object
dtypes: int64(7), object(1)
memory usage: 200.0+ KB


In [20]:
def separate_concatenated_words(text):
    if isinstance(text, str):
        # Split concatenated words with white space
        words = re.findall(r'\w+', text)
        separated_words = [re.sub('([a-z])([A-Z])', r'\1 \2', word) for word in words]
        return ' '.join(separated_words)
    else:
        return text

In [21]:
df['Full job']=df['Full job'].apply(separate_concatenated_words)

In [22]:
def preprocess_text(text):
    if isinstance(text, str):
        # Remove any URLs or email addresses
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        text = re.sub(r'\S+@\S+', '', text)

        # Rea move any sequence of alphanumeric characters starting with 'URL'
        text = re.sub(r'URL\w+', '', text)

        # Remove any non-alphanumeric characters
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

        # Split the text into sentences
        sentences = sent_tokenize(text)

        preprocessed_sentences = []
        for sentence in sentences:
            words = nltk.word_tokenize(sentence)

            # Remove any word contains numbers
            words = [word for word in words if not re.match('\w*\d\w*', word)]

            # Remove any word that is a stop word except "not", "Not", "IT", and "it"
            stop_words = set(nltk.corpus.stopwords.words('english'))
            stop_words.discard('not')
            stop_words.discard('Not')
            stop_words.discard('it')
            stop_words.discard('IT')
            words = [word for word in words if word.lower() not in stop_words or word.lower() in {'not', 'it'}]

            preprocessed_sentence = ' '.join(words)
            preprocessed_sentences.append(preprocessed_sentence)

        # Join the preprocessed sentences back together into a single string
        cleaned_text = ' '.join(preprocessed_sentences)

        return cleaned_text
    else:
        return text


In [23]:
df['Full job']= df['Full job'].apply(preprocess_text)

In [24]:
def replace_it(df, column_name):
    df[column_name] = df[column_name].apply(lambda x: re.sub(r'\b(it|It)\b', '', x) if isinstance(x, str) else x)
    df[column_name] = df[column_name].apply(lambda x: re.sub(r'\bIT\b', 'IT', x) if isinstance(x, str) else x)
    return df

In [25]:
#check if it's a noun then keep it else it's a stopword therefore it will be removed
df = replace_it(df, 'Full job')

In [26]:
def remove_extra_whitespace(text):
    if isinstance(text, str):
        # Replace one or more whitespace characters with a single space
        text = re.sub(r'\s+', ' ', text)

        return text.strip()  # Remove leading and trailing whitespace
    else:
        return text


In [27]:
df = df.applymap(remove_extra_whitespace)

In [28]:
def lowercase_text(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.lower()
    return df

In [29]:
df = lowercase_text(df)  

In [30]:
def remove_duplicates(text):
    if isinstance(text, list):
        word_list = text
    elif isinstance(text, str):
        word_list = text.split()
    else:
        return text
    
    unique_words = []
    for i, word in enumerate(word_list):
        if word not in unique_words:
            unique_words.append(word)
        elif i == 0:
            continue
        else:
            unique_words[-1] += ' ' + word
    
    if isinstance(text, list):
        return unique_words
    elif isinstance(text, str):
        return ' '.join(unique_words)


In [31]:
df['Full job'] = df['Full job'].apply(remove_duplicates)

In [32]:
df = df.applymap(remove_extra_whitespace)

In [33]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    if isinstance(text, str):
        # Split the text into sentences
        sentences = sent_tokenize(text)

        # Lemmatize each word in each sentence
        lemmatized_sentences = []
        for sentence in sentences:
            words = nltk.word_tokenize(sentence)
            lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
            lemmatized_sentences.append(' '.join(lemmatized_words))

        # Join the lemmatized sentences back together into a single string
        lemmatized_text = ' '.join(lemmatized_sentences)
        return lemmatized_text
    else:
        return text


In [34]:
df['Full job'] = df['Full job'].apply(lemmatize_text)

In [35]:
def remove_nonsense_words(text):
    # Define a list of regular expressions to match non-sensical words
    nonsense_patterns = [
        r'ou{2,}',  # Match "ou" followed by 2 or more "u"s
        r'[bcdfghjklmnpqrstvwxyz]{4,}',  # Match any sequence of 4 or more consonants
        r'[aeiouy]{4,}',  # Match any sequence of 4 or more vowels
        r'[^\x00-\x7F]+'  # Match any non-ASCII characters
        r'[a-z]*(wi){3,}[a-z]*',        # Match any word with "wi" repeated 3 or more times
        r'[a-z]*(x{n}o)+[a-z]*',        # Match any word with "xno" repeated one or more times
        r'[a-z]*(e{2,})+[a-z]*',        # Match any word with "ee" repeated two or more times
        r'n{2,}(e{2,})+[jkn]+',          # Match any word with "n" repeated two or more times followed by "e" repeated two or more times followed by one or more of "j", "k", or "n"
        r'\b\w*(\w)\1{2,}\w*\b'  # Match any word containing the same character repeated 3 or more times consecutively
    ]

    # Combine the patterns into a single regular expression
    pattern = '|'.join(nonsense_patterns)

    # Use the regular expression to remove the nonsense words
    return re.sub(pattern, '', text)


In [36]:
df['Full job'] = df['Full job'].apply(remove_nonsense_words)

In [42]:
df = df.rename(columns={'Full job': 'Job Info.'})

In [43]:
# replace 'indu' with 'industry' in the 'experience' column
df['Job Info.'] = df['Job Info.'].str.replace('indu', 'industry')

In [44]:
def remove_single_letters(text):
    # split the text into words
    words = text.split()

    # loop over the words
    for i in range(len(words)):
        # loop over the letters in the word
        j = 0
        while j < len(words[i]):
            if len(words[i]) == 1 and words[i][j] != 'e':
                # if the word has only one letter and it's not 'e', remove it
                words[i] = ''
                break
            else:
                j += 1

    # join the words back into a string
    new_text = ' '.join(words)

    return new_text

In [45]:
# apply the function to the 'experience' column
df['Job Info.'] = df['Job Info.'].apply(remove_single_letters)

In [46]:
# define the function to concatenate "e" with the next word
def concatenate_e(words):
    new_words = []
    for i in range(len(words)):
        if words[i] == "e" and i < len(words) - 1:
            new_words.append(words[i] + words[i+1])
        elif i > 0 and words[i-1] == "e":
            continue
        else:
            new_words.append(words[i])
    new_text = " ".join(new_words)
    return new_text

In [47]:
df['Job Info.'] = df['Job Info.'].apply(lambda x: concatenate_e(x.split()))

In [48]:
import re
from nltk.corpus import stopwords 
import string 
from nltk.stem import WordNetLemmatizer

from sklearn.ensemble import RandomForestClassifier ,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier ,ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score ,recall_score,precision_score,f1_score ,classification_report, r2_score ,confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stop words


    Error loading stop: Package 'stop' not found in index
      Package words is already up-to-date!



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [56]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [49]:
class TextPreprocessing() :
    def __init__(self, df : pd.DataFrame = pd.DataFrame) :
        self.df = df
        
        
    def Clean(self , df) :
        self.df = df
        df_copy = df.copy(deep = True)
        text_cols = list(df_copy.select_dtypes(include = ['object']).columns)
        for col in text_cols :
            for idx, text in enumerate(df_copy[col]) :
                te = []
                word = re.sub(r'(@|#)\w+' , '' , text)
                word = re.sub("[,.]", "", word)
                word = re.sub(r'https?://\S+' , '' , word)
                word = re.sub(r'(\?|!)+' , '' , word)
                word = re.sub(r"\(|\)", "", word)
                word = re.sub(r'(^\s+)' , '' , word)
                word = re.sub(r'(\s+$)' , '' , word)
                word = re.sub(r'\d+' , '' , word)
                word = word.split()
                for i in word :
                    if (i not in s_words) & (i not in punc) :
                        i = i.lower()
                        i = lmt.lemmatize(i , 'v')
                        te.append(i)
                df_copy.at[idx , col] = te
        return df_copy
    
    def Vactorize (self, df , target_name) :
        self.df = df
        self.target_name = target_name
        df_cleaned = df.copy(deep = True)
        text_cols = list(df_cleaned.select_dtypes(include = ['object']).columns)
        pos_word = {}
        neg_word = {}

        pos_df = df_cleaned[df_cleaned[target_name] == 1].reset_index(drop=True)
        neg_df = df_cleaned[df_cleaned[target_name] == 0].reset_index(drop=True)
        
        for col in text_cols :

            pos_word[col] = [word for sublist in pos_df[col] for word in sublist]
            neg_word[col] = [word for sublist in neg_df[col] for word in sublist]



        pos_freq = {}
        neg_freq = {}
        for key in pos_word.keys() :
            positive_dict = {}
            for word in pos_word[key] :
                positive_dict[word] = positive_dict.get(word , 0) + 1

            pos_freq[key] = positive_dict


        for key in neg_word.keys() :
            negative_dict = {}
            for word in neg_word[key] :
                negative_dict[word] = negative_dict.get(word , 0) + 1

            neg_freq[key] = negative_dict
            
        return pos_freq , neg_freq

        
        
        
    def Vactorization (self , df , target_name) :
        self.df = df
        self.target_name = target_name
        df_cleaned = df.copy(deep = True)
        text_cols = list(df_cleaned.select_dtypes(include = ['object']).columns)
        pos_freq , neg_freq = TextPreprocessing().Vactorize(df_cleaned , target_name)

        for col in text_cols :
            df_cleaned['{}_pos'.format(col)] = 0
            df_cleaned['{}_neg'.format(col)] = 0
            for idx, List in enumerate(df_cleaned[col]) :
                pos_frequent = 0
                neg_frequent = 0
                for word in List :
                    pos_frequent += pos_freq[col].get(word , 0)
                    neg_frequent += neg_freq[col].get(word , 0)



                df_cleaned.at[idx ,'{}_pos'.format(col)] = pos_frequent
                df_cleaned.at[idx ,'{}_neg'.format(col)] = neg_frequent
            df_cleaned.drop([col] , axis = 1 , inplace = True)
        return df_cleaned
    
    
    def fit_transform(self , df , target_name) :
        self.df = df
        self.target_name = target_name
        
        df_cleaned = TextPreprocessing().Clean(df)
        df_vact = TextPreprocessing().Vactorization(df_cleaned , target_name)
        
        return df_vact
    
    
    def Naive_Bayes(self , df, target_name) :
        self.df = df
        self.target_name = target_name
        df_naive = TextPreprocessing().Clean(df)
        pos_freq , neg_freq = TextPreprocessing().Vactorize(df_naive , target_name)
        text_cols = list(df_naive.select_dtypes(include = ['object']).columns)
        
        v_n_pos , v_n_neg = {} , {}

        for key in pos_freq.keys() :
            v_n_pos[key] = len(pos_freq[key])
            n = 0
            for word in pos_freq[key] :
                n += pos_freq[key].get(word , 0)
            v_n_pos[key]+=n
        for key in neg_freq.keys() :
            v_n_neg[key] = len(neg_freq[key])
            n = 0
            for word in neg_freq[key] :
                n += neg_freq[key].get(word , 0)
            v_n_neg[key]+=n
            
            
        prob_pos_dict = {}
        for key in pos_freq.keys():
            positive_dict = {}
            for word in pos_freq[key] :
                positive_dict[word] = (pos_freq[key].get(word , 0) + 1) / (v_n_pos[key])
            prob_pos_dict[key] = positive_dict




        prob_neg_dict = {}
        for key in neg_freq.keys():
            negative_dict = {}
            for word in neg_freq[key] :
                negative_dict[word] = (neg_freq[key].get(word , 0) + 1) / (v_n_neg[key])
                prob_neg_dict[key] = negative_dict
            
            
            
        for col in text_cols :
            df_naive['{}_probs'.format(col)] = 0
            for idx, List in enumerate(df_naive[col]) :
                score = 0
                for word in List :
                    try :
                        b = np.log((prob_pos_dict[col].get(word , 0)) / (prob_neg_dict[col].get(word , 0)))
                        if b == -float('inf') :
                            pass
                        else :
                            score +=b
                    except :
                        pass

                df_naive['{}_probs'.format(col)][idx] = score
            df_naive.drop([col] , axis = 1 , inplace = True)
            
            
        return {'probs_pos':prob_pos_dict ,'probs_neg':prob_neg_dict } , df_naive   

In [50]:
pre = TextPreprocessing()

In [51]:
df_cleaned = pre.Vactorization(df , target_name = 'fake?')
df_cleaned

,telecommuting,company logo exist?,has_questions,employment_type,experience required,education required,fake?,Job Info._pos,Job Info._neg
0,0,0,1,3,0,9,0,22517308,521189059
1,0,1,1,3,3,9,0,53771368,1246644048
2,0,1,0,3,4,9,0,37646325,872397379
3,0,1,0,3,0,9,0,10072483,233251791
4,0,1,1,3,0,9,0,51139977,1185952871
...,...,...,...,...,...,...,...,...,...
3193,0,1,1,3,2,9,0,64464732,1492997377
3194,0,0,1,0,0,9,1,28833985,667330929
3195,0,1,1,3,2,4,0,42238426,979752826
3196,0,0,0,3,3,9,0,55847681,1294107418


In [52]:
x = df_cleaned.drop(['fake?'] , axis = 1).values
y = df_cleaned['fake?'].values


In [53]:
pickled_model = pickle.load(open('RF_model_CCS', 'rb'))

In [58]:
y_pred = pickled_model.predict(x)

print("accuracy of RF': {}".format(accuracy_score(y_pred,y)))
print(metrics.classification_report(y,y_pred))

accuracy of RF': 0.9115071919949969
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      3030
           1       0.26      0.38      0.31       168

    accuracy                           0.91      3198
   macro avg       0.61      0.66      0.63      3198
weighted avg       0.93      0.91      0.92      3198



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(



# Credits

---



**Directed By:**

Member 1: [Eslam Shouman](https://www.linkedin.com/in/eslam-shouman-0958091b4/)

Member 2: [Rawan ibrahim](https://www.linkedin.com/in/rawan-ibrahim-a01778268) 

Member 3: [Mohamed Mahmoud](https://www.linkedin.com/in/mohamed-mahmoud07) 


**Date of creation:**

- **29 April 2023**